# Making the trainig sample from CANDELS GOODS-S 

By Shooby, last edited May29 2019



In [16]:
import numpy as np
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import astropy.wcs as wcs
from astropy import units as u
from astropy.coordinates import SkyCoord
from scipy import ndimage
from PIL import Image


%matplotlib inline

In [17]:
gs = pyfits.getdata('../../WFIRST_WPS/CANDELS_fits/gds.fits')

In [62]:
print(gs['B_IMAGE_1']/gs['A_IMAGE_1'])

[0.36268139 0.2033779  0.54978749 ... 0.41029523 0.91803279 0.81466877]


In [75]:
sel1 = (gs['zbest']>0.1)&(gs['zbest']<1.5)&(gs['CLASS_STAR']<0.85)&(gs['Hmag']<25)&(gs['B_IMAGE_1']/gs['A_IMAGE_1']<0.76)
sel2 = (gs['zbest']>0.1)&(gs['zbest']<1.5)&(gs['CLASS_STAR']<0.85)&(gs['Hmag']<25)&(gs['B_IMAGE_1']/gs['A_IMAGE_1']>0.76)

print (len(gs[sel1]),len(gs[sel2]))

3208 3280


In [76]:
def radec2xy(ra,dec,wc):
    coords = SkyCoord(ra,dec, unit='deg')
    a=wcs.utils.skycoord_to_pixel(coords, wc, origin=0,mode=u'wcs')
    return a[0],a[1]
    
def cut(ra,dec,andaze,filename):
    '''gets coordinates of the galaxy and the filter to return a cutout
    (also called a postage stamp) of the galaxy with given size'''
    hdr = pyfits.getheader(filename)
    w = wcs.WCS(hdr)
    x,y=radec2xy(ra,dec,w)
    x,y=np.int(x),np.int(y)
    im=pyfits.getdata(filename)[y-andaze:y+andaze,x-andaze:x+andaze]
    return im

def segment(a1,above_mean=1.1,add_image_back=0.3,random_noise=0.2):
    '''Returns segmentation of an image by some filtering of lower level 
    noises, to restrict the fitting to the galaxy itself and mask the 
    surroundings'''
    im2=ndimage.gaussian_filter(a1, 2)
    mask = (im2 > above_mean*im2.mean()).astype(np.float)
    mask += add_image_back * im2
    img = mask + random_noise*np.random.randn(*mask.shape)
    hist, bin_edges = np.histogram(img, bins=60)
    bin_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])
    binary_im = img > random_noise
    open_img = ndimage.binary_opening(binary_im)
    close_img = ndimage.binary_closing(open_img)
    return close_img

In [77]:
ra1,dec1=gs['RA_1'][sel1],gs['DEC_1'][sel1]
ra2,dec2=gs['RA_1'][sel2],gs['DEC_1'][sel2]

im = np.zeros([28,28])
for boz in range(3000):
    data = cut(ra1[boz],dec1[boz],14,'/Users/shemmati/Desktop/GOODS/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    data = np.tanh(data)
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/training_images/1/'+str(boz)+'.jpg')
    
    data = cut(ra2[boz],dec2[boz],14,'/Users/shemmati/Desktop/GOODS/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/training_images/2/'+str(boz)+'.jpg')
       
    #stop
    

In [78]:
ra1,dec1=gs['RA_1'][sel1],gs['DEC_1'][sel1]
ra2,dec2=gs['RA_1'][sel2],gs['DEC_1'][sel2]

im = np.zeros([28,28])
for boz in range(3000,3200):
    data = cut(ra1[boz],dec1[boz],14,'/Users/shemmati/Desktop/GOODS/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    data = np.tanh(data)
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/test_images/1/'+str(boz)+'.jpg')
    
    data = cut(ra2[boz],dec2[boz],14,'/Users/shemmati/Desktop/GOODS/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    rescaled = (255.0 / (data.max()+0.1) * (data - data.min())).astype(np.uint8)
    im = Image.fromarray(rescaled)
    im.save('images/test_images/2/'+str(boz)+'.jpg')
       
    #stop
    